In [ ]:
import pandas as pd
# from fastparquet import ParquetFile
import warnings
warnings.filterwarnings('ignore')
import datetime as dt
import os
import numpy as np
# from sqlalchemy import create_engine

import io
import time
import pandas as pd
import os

import numpy as np

from io import StringIO
import glob
import pandas as pd
#import win32com.client
import os
import glob
from pathlib import Path  # Standard Python Module
from openpyxl import load_workbook, Workbook  # pip install openpyxl
import glob
import ssl
import math
from scipy.stats import linregress
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import dateutil.relativedelta
from tqdm import tqdm
###Para cambiar los nombres de los meses a español
import locale
#locale.setlocale(locale.LC_TIME, 'es_ES')
import warnings
import pandas.io.sql as sqlio
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

filepath = '/content/drive/MyDrive/NuclioDSC/TFM/datasets_TFM/'

Mounted at /content/drive


In [ ]:
#Leamos los datasets y los almacenamos en las respectivas variables

productos = pd.read_csv(filepath + "product_ID_df.csv")
productos = productos.iloc[:,1:]

demografica = pd.read_csv(filepath + 'sociodemographic_df.csv')
demografica = demografica.iloc[:,1:]

comercial = pd.read_csv(filepath + 'commercial_activity_df.csv')
comercial = comercial.iloc[:, 1:]

ventas = pd.read_csv(filepath + 'sales_20_19_NEW.csv')
ventas = ventas.iloc[:, 1:]

tenencia = pd.read_csv(filepath + 'products_df.csv')
tenencia = tenencia.iloc[:, 1:]

## EDA-Data Cleaning

### Productos

In [ ]:
#Observamos el tamaño del dataframe
productos.shape

(13, 3)

In [ ]:
#Obtenemos información acerca de él
productos.info

<bound method DataFrame.info of     product_ID        product_desc family_product
0         4657           em_acount        account
1         3819          debit_card   payment_card
2         1364        pension_plan   pension_plan
3         2234             payroll        account
4         2235     payroll_account        account
5         8871         emc_account        account
6         2312         credit_card   payment_card
7         2335  short_term_deposit     investment
8         2336   long_term_deposit     investment
9         2673          securities     investment
10        1119               funds     investment
11        9001               loans           loan
12        4033            mortgage           loan>

In [ ]:
#Vemos el head del dataframe
productos.head(10)

,product_ID,product_desc,family_product
0,4657,em_acount,account
1,3819,debit_card,payment_card
2,1364,pension_plan,pension_plan
3,2234,payroll,account
4,2235,payroll_account,account
5,8871,emc_account,account
6,2312,credit_card,payment_card
7,2335,short_term_deposit,investment
8,2336,long_term_deposit,investment
9,2673,securities,investment


## Demográfica

In [ ]:
#Obtenemos el tamaño del dataframe
demografica.shape

(5962924, 8)

In [ ]:
#Obtenemos información acerca de él
demografica.info

<bound method DataFrame.info of           pk_cid pk_partition country_id  region_code gender  age deceased  \
0        1375586      2018-01         ES         29.0      H   35        N   
1        1050611      2018-01         ES         13.0      V   23        N   
2        1050612      2018-01         ES         13.0      V   23        N   
3        1050613      2018-01         ES         50.0      H   22        N   
4        1050614      2018-01         ES         50.0      V   23        N   
...          ...          ...        ...          ...    ...  ...      ...   
5962919  1166765      2019-05         ES         50.0      V   22        N   
5962920  1166764      2019-05         ES         26.0      V   23        N   
5962921  1166763      2019-05         ES         50.0      H   47        N   
5962922  1166789      2019-05         ES         50.0      H   22        N   
5962923  1550586      2019-05         ES         28.0      H   37        N   

            salary  
0         

In [ ]:
demografica.head(10)

,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary
0,1375586,2018-01,ES,29.0,H,35,N,87218.10
1,1050611,2018-01,ES,13.0,V,23,N,35548.74
2,1050612,2018-01,ES,13.0,V,23,N,122179.11
3,1050613,2018-01,ES,50.0,H,22,N,119775.54
4,1050614,2018-01,ES,50.0,V,23,N,NaN
5,1050615,2018-01,ES,45.0,H,23,N,22220.04
6,1050616,2018-01,ES,24.0,H,23,N,295590.36
7,1050617,2018-01,ES,50.0,H,23,N,113316.66
8,1050619,2018-01,ES,20.0,H,24,N,NaN
9,1050620,2018-01,ES,10.0,H,23,N,113194.98


In [ ]:
#Eliminamos las filas con valores faltantes
demografica.dropna(subset = ['pk_cid', 'pk_partition', 'country_id','gender', 'age',
       'deceased'],inplace = True)

#Seleccionamos solo las filas de demografica que corresponden a clientes que están vivos
lista_muertos =  demografica[demografica['deceased'] == "S"]['pk_cid'].values
demografica =demografica[~demografica['pk_cid'].isin(lista_muertos)]

#Pasando a categóricos la variavle gender a través del campo gnd
demografica['gnd'] = np.where(demografica['gender']== 'H',1,0)

In [ ]:
#Transformación categórica country_region ('ct_rgn')

#Unimos los campos 'country_id' con 'region_code'
demografica['ct_rgn'] = demografica['country_id'] + '_' + demografica['region_code'].astype(str)

#Obtenemos categorias asignadas a la variable 'ct_rgn'
numbers, categories = pd.factorize(demografica['ct_rgn'])

#Normalizamos los números asignados para obtener valores entre 0 y 1 (peso específico de cada categoria)
numbers_norm = numbers / (len(categories) - 1)
df_referencia_ct_rgn = demografica[['ct_rgn']]
df_referencia_ct_rgn['ct_rgn_norm'] = numbers_norm
df_referencia_ct_rgn.drop_duplicates(['ct_rgn_norm'],inplace=True)
demografica['ct_rgn_norm'] = numbers_norm
df_referencia_ct_rgn.reset_index(drop= True,inplace = True)

In [ ]:
demografica['pk_partition'] = pd.to_datetime(demografica['pk_partition'])


In [ ]:
demografica.head(2)

In [ ]:
temp_dem = demografica[['pk_cid','pk_partition','age','gnd','ct_rgn_norm']]
temp_dem.info()

### Comercial

In [ ]:
comercial.head(2)

In [ ]:
comercial.dropna(inplace=True)
counts = comercial['entry_channel'].value_counts()
weights = counts / len(comercial)
weights_dict = weights.to_dict()
comercial['entry_weights'] = comercial['entry_channel'].map(weights_dict)
df_referencia_entry_weights = comercial[['entry_channel','entry_weights']]
df_referencia_entry_weights.drop_duplicates(['entry_weights'],inplace=True)
comercial['pk_partition'] = pd.to_datetime(comercial['pk_partition'])
comercial['entry_date'] = pd.to_datetime(comercial['entry_date'])
comercial['anti_cli'] = (comercial['pk_partition'].dt.year - comercial['entry_date'].dt.year) * 12 + (comercial['pk_partition'].dt.month - comercial['entry_date'].dt.month)
comercial['seg_cli'] = comercial['segment'].str.split("-",expand=True)[0]

In [ ]:
comercial['anti_cli'] = comercial['anti_cli'].astype(float)
comercial['seg_cli'] = comercial['seg_cli'].astype(int)
comercial.info()

In [ ]:
comercial = comercial[['pk_cid', 'pk_partition','entry_weights', 'anti_cli', 'seg_cli']]

In [ ]:
comercial.drop_duplicates(['pk_cid', 'pk_partition','entry_weights', 'anti_cli', 'seg_cli'],inplace = True)


In [ ]:
comercial.head(2)

In [ ]:
temp_com = comercial[['pk_cid', 'pk_partition','entry_weights', 'anti_cli', 'seg_cli']]
temp_com['pk_partition'] = pd.to_datetime(temp_com['pk_partition'],format = '%Y-%m')

<h2><u>VENTAS<u><h2>

In [ ]:
ventas.head(2)

In [ ]:
ventas.drop_duplicates(['pk_sale', 'cid', 'month_sale', 'product_ID', 'net_margin'],inplace = True)

In [ ]:
len(ventas['pk_sale'].unique())-len(ventas)

In [ ]:
ventas['target'] = np.where(ventas['net_margin']>0,1,0)
ventas.sort_values(['net_margin'],ascending=False)
ventas.rename(columns={'month_sale':'pk_partition'}, inplace=True)
ventas.rename(columns={'cid':'pk_cid'}, inplace=True)
ventas['pk_partition'] = pd.to_datetime(ventas['pk_partition'],dayfirst = True)

In [ ]:
temp_ventas = ventas

In [ ]:
temp_ventas.head(2)

In [ ]:
temp_ventas.info()

<h2><u>TENENCIA DE PRODUCTOS<u><h2>

<h2><u>MERGE DE TABLAS<u><h2>

In [ ]:
df = temp_dem.merge(temp_com,on = ['pk_cid','pk_partition'],how = 'left')
df = df.merge(temp_ventas,on = ['pk_cid','pk_partition'],how = 'left')
df.sort_values(['target'],inplace = True,ascending=True)
df.drop_duplicates(['pk_cid','pk_partition'],inplace=True)

In [ ]:
df

In [ ]:
df['target'].value_counts()[1]

In [ ]:
df['target'] = np.where(df['target'].isnull(),0,df['target'])

In [ ]:
df

In [ ]:
df.fillna(0, inplace=True)

In [ ]:
df

In [ ]:
df = df.sort_values(by=['pk_cid', 'pk_partition'])

# Agregar nueva columna TARGET_1
df['TARGET_1'] = df.groupby('pk_cid')['target'].shift(1)



In [ ]:
df

In [ ]:
df.columns

In [ ]:
df.dropna(subset = ['TARGET_1'],inplace=True) #Elimino aquellos casos que no cuentan con información el mes anterior no compraron y no registran actividad comercial


In [ ]:
df=df[['pk_cid', 'pk_partition', 'age', 'gnd', 'ct_rgn_norm', 'entry_weights','anti_cli', 'seg_cli','target','TARGET_1']]

In [ ]:
merge_target=df

In [ ]:
merge_target['target'].value_counts()[1]

In [ ]:
cant_mes1= merge_target.groupby(['pk_partition','target'])['target'].count().unstack()
cant_mes1


In [ ]:
cant_mes2= merge_target.groupby(['pk_partition','target'])['target'].count().unstack()
cant_mes2

In [ ]:
##preparar 70 30%
casos = merge_target[merge_target['pk_partition']=='2018-09-01']
casos_1= casos[casos['target']==1]
casos_0 = casos[casos['target']==0]
casos_0_sample = casos_0.sample(int(len(casos_1)*1),random_state=1)
casos_modelo_60 = pd.concat([casos_1,casos_0_sample])
len(casos_modelo_60)-len(casos_1)-len(casos_0_sample)

In [ ]:
cant_mes2= casos_modelo_60.groupby(['pk_partition','target'])['target'].count().unstack()
cant_mes2

In [ ]:
casos_modelo_60

In [ ]:
muestra=casos_modelo_60

# **ENTRENAMIENTO**

In [ ]:
df_modelo =muestra
df_modelo = df_modelo.iloc[:,2:]

In [ ]:
df_modelo

In [ ]:
len(df_modelo)

In [ ]:
!pip install yellowbrick==1.3.1

In [ ]:
!pip install numpy==1.20


In [ ]:
!pip install pycaret

In [ ]:
!pip install pycaret
from pycaret.classification import *
s = setup(df_modelo, target = 'target')


In [ ]:
best = compare_models()

In [ ]:
evaluate_model(best)

In [ ]:
save_model(best, 'model_train')

# PREDICCION

In [ ]:
# Modificando formatos fecha
tenencia['pk_partition'] = pd.to_datetime(tenencia['pk_partition'])

In [ ]:
# Cambiando el formato de pk_partition de la tabla tendencia
tenencia['pk_partition'] = pd.to_datetime(tenencia['pk_partition'],dayfirst = True)
tenencia['pk_partition'] = tenencia['pk_partition'].dt.year.astype(str) +'-'+tenencia['pk_partition'].dt.month.astype(str) +'-'+ '01'
tenencia['pk_partition'] = pd.to_datetime(tenencia['pk_partition'],dayfirst = True)

In [ ]:
tenencia1=tenencia[tenencia.pk_partition== '2019-05-01']

In [ ]:
fecha_mes_anterior = pd.Timestamp('2019-04-01')
tenencia1['fecha_mes_anterior'] =fecha_mes_anterior

In [ ]:
tenencia1=tenencia1[['pk_cid', 'pk_partition','fecha_mes_anterior']]

In [ ]:
ventas['fecha_mes_anterior'] = ventas['pk_partition']###.astype(int)
data_predict=tenencia1.merge(ventas,on=['pk_cid','pk_partition'], how='left')

In [ ]:
data_predict=data_predict.merge(productos,on=['product_ID'], how='left')

In [ ]:
data_predict

In [ ]:
data_predict = data_predict.fillna(0)

In [ ]:
data_predict.columns

In [ ]:
df_prediccion = data_predict
df_prediccion.reset_index(inplace = True,drop = True)

In [ ]:
cant_mes1=df_prediccion.groupby(['pk_partition','product_desc'])['product_desc'].count().unstack()
cant_mes1

PRODUCTO : EM_ACOUNT

In [ ]:
##preparar Predicción
casos = df_prediccion
casos_1= casos[casos['product_desc']=="em_acount"]
casos_0 = casos[casos['product_desc']==0]

In [ ]:
# crear una muestra aleatoria del dataframe casos_0 con un tamaño del X% del número de filas en el dataframe casos_1
casos_0_sample = casos_0.sample(int(len(casos_1)*1),random_state=1)
casos_0_sample

In [ ]:
casos_modelo = pd.concat([casos_1,casos_0_sample])
len(casos_modelo)-len(casos_1)-len(casos_0_sample)

In [ ]:
casos_modelo

In [ ]:
muestra['pk_cid']=muestra['pk_cid'].astype(int)
casos_modelo['pk_cid']=casos_modelo['pk_cid'].astype(int)

In [ ]:
casos_modelo=casos_modelo.merge(muestra,on=['pk_cid'], how='left')
casos_modelo

In [ ]:
casos_modelo = casos_modelo.fillna(0)

In [ ]:
casos_modelo1_cli=casos_modelo[['pk_cid']]
casos_modelo1=casos_modelo[['age', 'gnd', 'ct_rgn_norm','entry_weights', 'anti_cli', 'seg_cli', 'TARGET_1' ]]
casos_modelo1

In [ ]:
df_modelo = casos_modelo1
len(df_modelo)

In [ ]:
df_modelo

In [ ]:
from pycaret.classification import load_model

# Load the trained model
model = load_model('/content/model_train')

# Use the model to make predictions
predictions = model.predict_proba(df_modelo)
predictions_1 = model.predict(df_modelo)

In [ ]:
df_modelo[['%_resid','%_prope']] = pd.DataFrame(predictions)


In [ ]:
df_modelo

In [ ]:
df_modelo_unido = casos_modelo1_cli.join(df_modelo)

In [ ]:
df_modelo_unido.head()

In [ ]:
##resultado = df.loc[df['pk_cid'] == 649919]

In [ ]:
df_modelo_unido = df_modelo_unido.sort_values(by='%_prope', ascending=False)
df_modelo_unido

In [ ]:
#percentiles = np.percentile(df_modelo_unido['%_prope'],[10, 20, 30, 40, 50, 60, 70, 80, 90,95])
#df_modelo_unido['percentil_compra']= np.searchsorted(percentiles,df_modelo_unido['%_prope'])

In [ ]:
# Calcular los percentiles
percentiles = np.percentile(df_modelo_unido['%_prope'], np.arange(0, 100, 10))

# Asignar percentiles a cada registro
df_modelo_unido['percentil_compra'] = np.searchsorted(percentiles, df_modelo_unido['%_prope'])

# Imprimir los primeros 10 registros para verificar
print(df_modelo_unido.head(10))

In [ ]:
df_modelo_unido=df_modelo_unido[['pk_cid','%_prope','percentil_compra']]

In [ ]:
df_modelo_unido.head()

In [ ]:
#df_modelo_unido.info()

In [ ]:
#df_modelo_unido.isna().sum()

In [ ]:
df_modelo_unido.to_excel('PROD_EM_ACOUNT.xlsx', index=False)